In [1]:
import pickle
import dotenv

In [2]:
import tqdm
import time
import re

In [3]:
from mmw.parse import *

In [4]:
dotenv.load_dotenv()

True

In [9]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [10]:
questions_explanation = parse_question_explanations()

In [11]:
all_parsed_essays = parse_essays()

In [12]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [13]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [14]:
model = "text-davinci-003"

#models simple/ cheap/ fast to advanced:
#model = "ada"
#model = "babbage"
#model = "curie"
#model = "davinci"

#model = "ada:ft-mark-my-words-2023-05-01-11-44-26"

In [55]:
prompt_tpl = 'You are a caring teacher who wants to help their students succeed by understanding their mistakes and how to improve. Given is a student essay and a graded remark to a question. Grades can be numbers between 0 (lowest, worst) and 3 (highest, best). Give a possible reason and detailed explanation for the graded remark along with a reference from the Essay. Use direct form to answer the student. Start with a positive feedback.\n\n###\n\n'
prompt_tpl += 'Essay:\n{essay}\n\n###\n\n'
prompt_tpl += 'Question:\n{explanation}\n{question}\n\n###\n\n'
prompt_tpl += 'Grade: {remark}\n\n###\n\n'
prompt_tpl += 'Answer:'

In [60]:
def assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation):

    all_prompts = []
    for i,essay_array in tqdm.tqdm(enumerate(all_parsed_essays)):

        essay_id = int(re.findall(r'\d+', essay_array[0])[-1])
        cur_row = filtered_marked_essays[filtered_marked_essays['essay id'] == essay_id]
        if len(cur_row) != 1:
            continue

        for j,k in questions_explanation.iterrows():
            question = k.question
            explanation = k.explanation
            grade = int(cur_row.iloc[0,j+1])
            prompt = prompt_tpl.format(essay=essay_array[3][:-1], #implicit newline
                                    question=question.strip(),
                                    explanation=explanation.strip(),
                                    remark=grade)

            all_prompts.append((i, j, prompt, grade))
    return all_prompts


In [61]:
all_prompts = assemble_citation_prompts(all_parsed_essays, filtered_marked_essays, questions_explanation)

77it [00:00, 532.87it/s]


In [52]:
LIMIT_PROMPTS = 1

In [62]:
all_answers = []
for prompt_array in tqdm.tqdm(all_prompts): #[:LIMIT_PROMPTS*len(questions_explanation)]):
    for i in range(3):
        try:
            i, j, prompt, annot = prompt_array
            response = openai.Completion.create(model=model, prompt=prompt, temperature=0, max_tokens=250)
            all_answers.append((i, j, prompt, response, annot))
            
            time.sleep(.5)
            break
        except:
            print(f"exception, retrying {i}")
            time.sleep(2.)

  0%|          | 0/988 [00:00<?, ?it/s]

exception, retrying 0
exception, retrying 0
exception, retrying 0


  0%|          | 1/988 [00:06<1:50:25,  6.71s/it]

exception, retrying 0


  0%|          | 2/988 [00:19<2:45:51, 10.09s/it]

...

100%|█████████▉| 984/988 [9:49:36<00:52, 13.10s/it]

exception, retrying 76


100%|█████████▉| 986/988 [9:50:05<00:27, 13.54s/it]

exception, retrying 76


100%|██████████| 988/988 [9:50:26<00:00, 35.86s/it]


In [63]:
save_results('data/evaluations/run_6_davinci_comments', all_parsed_essays, all_questions, all_answers)

In [6]:
_,_,all_answers_loaded = load_results('data/evaluations/run_4_text-davinci_explanation')

In [8]:
all_answers_loaded[0][2]

'Given is a student essay and a question to the essay. Please rate the Question with a number between 0 and 3, where 0 is "not at all satisfied" and 3 is "very satisfied.".\n\nEssay: """\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time forwords and

In [7]:
[(a[0], a[1], a[-1], a[3]['choices'][0]['text']) for a in all_answers_loaded]

[(0, 0, 1, ' 3'),
 (0, 1, 1, ' 3'),
 (0, 2, 1, ' 3'),
 (0, 3, 1, ' 3'),
 (0, 4, 1, ' 3'),
 (0, 5, 0, ' 3'),
 (0, 6, 0, ' 3'),
 (0, 7, 0, ' 3'),
 (0, 8, 0, ' 3'),
 (0, 9, 0, ' 3'),
 (0, 10, 0, ' 3'),
 (0, 11, 0, ' 3'),
 (0, 12, 0, ' 3'),
 (1, 0, 2, ' 3'),
 (1, 1, 2, ' 3'),
 (1, 2, 3, ' 3'),
 (1, 3, 3, ' 3'),
 (1, 4, 2, ' 3'),
 (1, 5, 3, ' 3'),
 (1, 6, 3, ' 3'),
 (1, 7, 2, ' 3'),
 (1, 8, 3, ' 3'),
 (1, 9, 2, ' 3'),
 (1, 10, 2, ' 3'),
 (1, 11, 3, ' 3'),
 (1, 12, 3, ' 3'),
 (2, 0, 3, ' 3'),
 (2, 1, 3, ' 3'),
 (2, 2, 3, ' 3'),
 (2, 3, 3, ' 3'),
 (2, 4, 3, ' 3'),
 (2, 5, 3, ' 3'),
 (2, 6, 3, ' 3'),
 (2, 7, 3, ' 3'),
 (2, 8, 3, ' 3'),
 (2, 9, 3, ' 3 - Very'),
 (2, 10, 3, ' 3'),
 (2, 11, 3, ' 3'),
 (2, 12, 1, ' 3'),
 (3, 0, 1, ' 3'),
 (3, 1, 3, ' 3'),
 (3, 2, 3, ' 3'),
 (3, 3, 3, ' 3'),
 (3, 4, 3, ' 3'),
 (3, 5, 3, ' 3'),
 (3, 6, 3, ' 3'),
 (3, 7, 3, ' 3'),
 (3, 8, 3, ' 3'),
 (3, 9, 3, ' 3'),
 (3, 10, 3, ' 3'),
 (3, 11, 3, ' 3'),
 (3, 12, 2, ' 3'),
 (4, 0, 2, ' 3'),
 (4, 1, 2, ' 3'),
 (4, 2, 2